In [1]:
import pandas as pd
import datetime
from minio import Minio

import sys
sys.path.append(r'D:\Documentos\Projetos\Ollist_End_to_End\airflow\dags')
import functions as f

In [3]:
data_lake_server= f.var['data_lake_server_local']
data_lake_login= f.var['data_lake_login']
data_lake_password= f.var['data_lake_password']

client = Minio(
        endpoint= data_lake_server,
        access_key= data_lake_login,
        secret_key= data_lake_password,
        secure=False
    )

In [10]:
objects = client.list_objects('landing')

for obj in objects:
    print(obj.object_name)

olist_customers_dataset.csv
olist_geolocation_dataset.csv
olist_order_items_dataset.csv
olist_order_payments_dataset.csv
olist_order_reviews_dataset.csv
olist_orders_dataset.csv
olist_products_dataset.csv
olist_sellers_dataset.csv


In [24]:
client = pd.read_csv(
    f.read_minio_file(bucket_name= 'landing', file= 'olist_customers_dataset.csv', client= client)
)

In [20]:
geo = pd.read_csv(
    f.read_minio_file(bucket_name= 'landing', file= 'olist_geolocation_dataset.csv', client= client)
)

In [30]:
client[['customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']].duplicated().sum()

3089

In [38]:
client.shape

(99441, 5)

In [42]:
client.merge(
    geo.groupby(
        ['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state']
    )[['geolocation_lat', 'geolocation_lng']].mean().reset_index(),
    left_on= ['customer_zip_code_prefix', 'customer_city', 'customer_state'],
    right_on= ['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state'],
    how= 'left').drop(columns= ['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state'])

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_lat,geolocation_lng
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,-20.498489,-47.396929
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,-23.728035,-46.542818
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,-23.531705,-46.656207
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,-23.499702,-46.185233
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,-22.975100,-47.142925
...,...,...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP,-23.585979,-46.499703
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP,-23.615877,-46.768522
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE,-3.734569,-38.510534
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS,-29.949839,-51.168494


In [32]:
geo.columns

Index(['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng',
       'geolocation_city', 'geolocation_state'],
      dtype='object')

In [39]:
geo.groupby(
    ['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state']
)[['geolocation_lat', 'geolocation_lng']].mean().reset_index()

,geolocation_zip_code_prefix,geolocation_city,geolocation_state,geolocation_lat,geolocation_lng
0,1001,sao paulo,SP,-23.550215,-46.634019
1,1001,são paulo,SP,-23.549998,-46.634060
2,1002,sao paulo,SP,-23.548438,-46.635129
3,1002,são paulo,SP,-23.544641,-46.633180
4,1003,sao paulo,SP,-23.548988,-46.635786
...,...,...,...,...,...
27907,99965,agua santa,RS,-28.183372,-52.039850
27908,99970,ciriaco,RS,-28.343838,-51.874623
27909,99970,ciríaco,RS,-28.342325,-51.876023
27910,99980,david canabarro,RS,-28.389129,-51.843836


In [1]:
import os
import glob

files_remove = glob.glob('tmp/*')

for f in files_remove:
    os.remove(f)